# Import knižníc

In [1]:
import pandas as pd
import numpy as np

from PIL import Image
import cv2
import glob 

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.colors import LightSource
from matplotlib import cbook, cm
from matplotlib.ticker import PercentFormatter

import threading
from multiprocessing import Process

# Všemožné prevádzačky a matematiky na dátumoch

In [2]:
def inp_to_date(year, month, day, hour, minute):
    year, month, day, hour, minute = str(year).zfill(4), str(month).zfill(2), str(day).zfill(2), str(hour).zfill(2), str(minute).zfill(2)
    return np.datetime64(f"{year}-{month}-{day}T{hour}:{minute}", "s")

In [3]:
def inp_to_img(year, month, day, hour, minute):
    year, month, day, hour, minute = str(year).zfill(4), str(month).zfill(2), str(day).zfill(2), str(hour).zfill(2), str(minute).zfill(2)
    return f"{year}{month}{day}_{hour}{minute}"

In [4]:
def img_to_date(date):
    year, month, day, hour, minute = date[0:4], date[4:6], date[6:8], date[9:11], date[11:13]
    return np.datetime64(f"{year}-{month}-{day}T{hour}:{minute}", "s")

In [5]:
def img_to_plot(date):
    year, month, day, hour, minute = date[0:4], date[4:6], date[6:8], date[9:11], date[11:13]
    return f"{year}-{month}-{day} {hour}:{minute}"

In [6]:
def date_to_img(date):
    temp = np.datetime_as_string(date)
    year, month, day, hour, minute = temp[0:4], temp[5:7], temp[8:10], temp[11:13], temp[14:16]
    return f"{year}{month}{day}_{hour}{minute}"

In [7]:
def dif_time(date1, date2):
    return np.abs((date1 - date2)).astype(int) #.item().total_seconds()) 

In [8]:
def dash_core(item_date):
    item_date = np.datetime_as_string(item_date)
    year, month, day, hour, minute = item_date[0:4], item_date[5:7], item_date[8:10], item_date[11:13], item_date[14:16]

    date = inp_to_date(year, month, day, hour, minute)
    start_of_day = inp_to_date(year, month, day, 0, 0) - np.timedelta64(10, 'D')
    end_of_day = inp_to_date(year, month, day, 0, 0) + np.timedelta64(10, 'D')

    eit_index = np.argmin(dif_time(date, eit_list))
    LC2_index = np.argmin(dif_time(date, LC2_list))
    mag_index = np.argmin(dif_time(date, mag_list))

    eit_date = date_to_img(eit_list[eit_index])
    LC2_date = date_to_img(LC2_list[LC2_index])
    mag_date = date_to_img(mag_list[mag_index])
    
    N_p_index = np.argmin(dif_time(date, np.array(N_p_list["Time"])))
    V_p_index = np.argmin(dif_time(date, np.array(V_p_list["Time"])))
    B_z_index = np.argmin(dif_time(date, np.array(B_z_list["Time"])))
    
    eit_path = f"{folder}/soho_eit195/{eit_date}_eit195_1024.png"
    LC2_path = f"{folder}/soho_lasco/c2/{LC2_date}_lascoc2_1024.png"
    mag_path = f"{folder}/soho_mdi/mag/{mag_date}_mdimag_1024.png"
    
    mag_img = Image.open(mag_path).convert('1')
    LC2_img = Image.open(LC2_path)
    eit_img = Image.open(eit_path)
    
    plt.rcParams.update({'font.size': 8})
    fig = plt.figure(figsize=(15, 10), layout='constrained', facecolor="black", edgecolor='white')
    axs = fig.subplot_mosaic([["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1"],
                              ["SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2"],
                              ["SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3"]])
    
    
    
    axs["IMG1"].imshow(mag_img)
    axs["IMG2"].imshow(eit_img)
    axs["IMG3"].imshow(LC2_img)
    
    for temp_img, temp_name in zip(["IMG1", "IMG2", "IMG3"], [str(img_to_plot(mag_date)), str(img_to_plot(eit_date)), str(img_to_plot(LC2_date))]):
        axs[temp_img].spines['bottom'].set_color('white')
        axs[temp_img].spines['top'].set_color('white')
        axs[temp_img].spines['left'].set_color('white')
        axs[temp_img].spines['right'].set_color('white')
        axs[temp_img].locator_params(axis='y', nbins=10) 
        axs[temp_img].locator_params(axis='x', nbins=10) 
        axs[temp_img].xaxis.set_minor_locator(AutoMinorLocator(10))
        axs[temp_img].yaxis.set_minor_locator(AutoMinorLocator(10))
        axs[temp_img].tick_params(which='major', length=6, direction="inout", colors="white")
        axs[temp_img].tick_params(which='minor', length=2, direction="in", colors="white") 
        axs[temp_img].set_title(temp_name, fontsize = 20)
        axs[temp_img].xaxis.label.set_color('white')
        axs[temp_img].yaxis.label.set_color('white')
        axs[temp_img].tick_params(colors='white', which='both') 
        axs[temp_img].title.set_color('white')
    
    axs["SERIES1"].plot(np.array(N_p_list.loc[N_p_list["Time"].between(start_of_day, end_of_day)]["Time"]), N_p_list.loc[N_p_list["Time"].between(start_of_day, end_of_day)]["N_p"], color="red", marker=".", linestyle="", markersize=1)
    axs["SERIES2"].plot(np.array(V_p_list.loc[V_p_list["Time"].between(start_of_day, end_of_day)]["Time"]), V_p_list.loc[V_p_list["Time"].between(start_of_day, end_of_day)]["V_p"], color="green", marker=".", linestyle="", markersize=1)
    axs["SERIES3"].plot(np.array(B_z_list.loc[B_z_list["Time"].between(start_of_day, end_of_day)]["Time"]), B_z_list.loc[B_z_list["Time"].between(start_of_day, end_of_day)]["B_z"], color="yellow", marker=".", linestyle="", markersize=1)
    
    for temp_series, temp_name in zip(["SERIES1", "SERIES2", "SERIES3"], ["rho", "v", "Bz"]):
        axs[temp_series].spines['bottom'].set_color('white')
        axs[temp_series].spines['top'].set_color('white')
        axs[temp_series].spines['left'].set_color('white')
        axs[temp_series].spines['right'].set_color('white')
        axs[temp_series].set_facecolor('black')
        axs[temp_series].locator_params(axis='y', nbins=5) 
        axs[temp_series].yaxis.set_minor_locator(AutoMinorLocator(5))
        axs[temp_series].tick_params(which='major', length=6, direction="inout", colors="white")
        axs[temp_series].tick_params(which='minor', length=2, direction="in", colors="white") 
        axs[temp_series].xaxis.label.set_color('white')
        axs[temp_series].yaxis.label.set_color('white')
        axs[temp_series].tick_params(colors='white', which='both') 
        axs[temp_series].title.set_color('white')
    
    axs['SERIES1'].set_ylabel(r'Density (cm$^{-3}$)')
    axs['SERIES2'].set_ylabel(r'Speed (cm $\cdot$ s$^{-1}$)')
    axs['SERIES3'].set_ylabel(r'Bz (nT)')
    
    axs["SERIES3"].set_xlabel('Date')
    axs['SERIES2'].sharex(axs['SERIES1'])
    axs['SERIES3'].sharex(axs['SERIES2'])
    
    axs["SERIES1"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')
    axs["SERIES2"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')
    axs["SERIES3"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')
    
    plt.savefig("Dash_" + date_to_img(date) + ".png", dpi=300, facecolor='black', edgecolor='black')
    plt.close()
    # plt.show()

In [14]:
folder = "2004-11-08"
csv_date = folder[0:4]+folder[5:7]+folder[8:10]

eit_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_eit195/*")])[::, 1]
eit_list= [img_to_date(temp) for temp in eit_list]

LC2_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_lasco/c2/*")])[::, 1]
LC2_list= [img_to_date(temp) for temp in LC2_list]

mag_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_mdi/mag/*")])[::, 1]
mag_list= [img_to_date(temp) for temp in mag_list]

path4 = f"{folder}/in_situ/{csv_date}_soho_N_p.csv"
path5 = f"{folder}/in_situ/{csv_date}_soho_V_p.csv"
path6 = f"{folder}/in_situ/{csv_date}_wind_B_z.csv"            

N_p_list = pd.read_csv(path4).drop(columns=['Unnamed: 0'])
V_p_list = pd.read_csv(path5).drop(columns=['Unnamed: 0'])
B_z_list = pd.read_csv(path6).drop(columns=['Unnamed: 0'])

N_p_list["Time"] = np.array(N_p_list["Time"]).astype("datetime64")
V_p_list["Time"] = np.array(V_p_list["Time"]).astype("datetime64")
B_z_list["Time"] = np.array(B_z_list["Time"]).astype("datetime64")

for item_date in np.arange(eit_list[0], eit_list[-1], dtype='datetime64[h]')[0:10]:
    dash_core(item_date)
